# Basic Fixed Income Trading Bot
Basic fixed income trading bot that trades TLT: iShares 20+ Year Treasury Bond ETF, MUB: iShares National Muni Bond ETF, LQD: iShares iBoxx $ Investment Grade Corporate Bond ETF, MBB: iShares MBS ETF with a moving average strategy, with 20% drawdown

In [30]:
pip install -r requirements.txt

You should consider upgrading via the '/Users/aryanlangeh/algo-trading-bot/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np

In [32]:
# Fetch data function
def fetch_data(ticker):
    asset = yf.Ticker(ticker)
    data = asset.history(period="1y")['Close']
    return data

In [33]:
# Fetch data
tickers = ['IEF', 'MBB', 'LQD']  # ETFs representing different fixed income classes
prices = {ticker: fetch_data(ticker) for ticker in tickers}

In [34]:
initial_balance = 100000  # Initial investment in USD
balance = initial_balance
portfolio = {ticker: 0 for ticker in tickers}
cash = initial_balance
trade_count = 0
winning_trades = 0

In [52]:
df = yf.download(tickers, start='2023-08-01', end='2023-08-28')


[*********************100%%**********************]  3 of 3 completed


In [53]:
# Strategy parameters
sma_short = 20
sma_long = 50
bollinger_window = 20
bollinger_num_std_dev = 2
macd_short = 12
macd_long = 26
macd_signal = 9

In [54]:
# Create DataFrame to hold signals
signals = {}

for ticker in tickers:
    signals[ticker] = pd.DataFrame(index=df.index)
    signals[ticker]['price'] = df['Adj Close'][ticker]
    print("Calculating signals for " + ticker)
    
    # Moving Averages
    signals[ticker]['sma_short'] = signals[ticker]['price'].rolling(window=sma_short).mean()
    signals[ticker]['sma_long'] = signals[ticker]['price'].rolling(window=sma_long).mean()
    print(signals[ticker]['sma_short'])
    print(signals[ticker]['sma_long'])

    # Bollinger Bands
    signals[ticker]['rolling_mean'] = signals[ticker]['price'].rolling(window=bollinger_window).mean()
    signals[ticker]['rolling_std'] = signals[ticker]['price'].rolling(window=bollinger_window).std()
    signals[ticker]['bollinger_upper'] = signals[ticker]['rolling_mean'] + (signals[ticker]['rolling_std'] * bollinger_num_std_dev)
    signals[ticker]['bollinger_lower'] = signals[ticker]['rolling_mean'] - (signals[ticker]['rolling_std'] * bollinger_num_std_dev)
    print(signals[ticker]['bollinger_upper'])
    print(signals[ticker]['bollinger_lower'])

    # MACD
    signals[ticker]['ema_short'] = signals[ticker]['price'].ewm(span=macd_short, adjust=False).mean()
    signals[ticker]['ema_long'] = signals[ticker]['price'].ewm(span=macd_long, adjust=False).mean()
    signals[ticker]['macd'] = signals[ticker]['ema_short'] - signals[ticker]['ema_long']
    signals[ticker]['signal_line'] = signals[ticker]['macd'].ewm(span=macd_signal, adjust=False).mean()
    print(signals[ticker]['macd'])
    print(signals[ticker]['signal_line'])

    # Create Buy and Sell signals
    signals[ticker]['positions'] = 0.0
    
    # Buy if price > sma_long and price > bollinger_lower and macd > signal_line
    buy_mask = (signals[ticker]['price'] > signals[ticker]['sma_long']) & \
               (signals[ticker]['price'] > signals[ticker]['bollinger_lower']) & \
               (signals[ticker]['macd'] > signals[ticker]['signal_line'])

    signals[ticker].loc[buy_mask, 'positions'] = 1.0
    
    # Sell if price < sma_long or price < bollinger_upper or macd < signal_line
    sell_mask = (signals[ticker]['price'] < signals[ticker]['sma_long']) | \
                (signals[ticker]['price'] < signals[ticker]['bollinger_upper']) | \
                (signals[ticker]['macd'] < signals[ticker]['signal_line'])
    
    signals[ticker].loc[sell_mask, 'positions'] = -1.0

Calculating signals for IEF
Date
2023-08-01   NaN
2023-08-02   NaN
2023-08-03   NaN
2023-08-04   NaN
2023-08-07   NaN
2023-08-08   NaN
2023-08-09   NaN
2023-08-10   NaN
2023-08-11   NaN
2023-08-14   NaN
2023-08-15   NaN
2023-08-16   NaN
2023-08-17   NaN
2023-08-18   NaN
2023-08-21   NaN
2023-08-22   NaN
2023-08-23   NaN
2023-08-24   NaN
2023-08-25   NaN
Name: sma_short, dtype: float64
Date
2023-08-01   NaN
2023-08-02   NaN
2023-08-03   NaN
2023-08-04   NaN
2023-08-07   NaN
2023-08-08   NaN
2023-08-09   NaN
2023-08-10   NaN
2023-08-11   NaN
2023-08-14   NaN
2023-08-15   NaN
2023-08-16   NaN
2023-08-17   NaN
2023-08-18   NaN
2023-08-21   NaN
2023-08-22   NaN
2023-08-23   NaN
2023-08-24   NaN
2023-08-25   NaN
Name: sma_long, dtype: float64
Date
2023-08-01   NaN
2023-08-02   NaN
2023-08-03   NaN
2023-08-04   NaN
2023-08-07   NaN
2023-08-08   NaN
2023-08-09   NaN
2023-08-10   NaN
2023-08-11   NaN
2023-08-14   NaN
2023-08-15   NaN
2023-08-16   NaN
2023-08-17   NaN
2023-08-18   NaN
2023-08-21

In [38]:
# Initialize additional variables
max_drawdown = 0.2  # 20% maximum allowable drawdown
fee_per_trade = 5  # $5 fee per trade

# Risk management variables
highest_balance = initial_balance  # Record of the highest account balance
drawdown_limit = highest_balance * max_drawdown  # Calculate the maximum allowable drawdown


In [44]:
# Backtesting
for i in range(1, len(df)):
    current_balance = cash
    for ticker in portfolio:
        current_balance += portfolio[ticker] * df['Adj Close'][ticker].iloc[i]

    # Update highest_balance if current_balance is higher
    highest_balance = max(highest_balance, current_balance)
    drawdown_limit = highest_balance * max_drawdown  # Update the maximum allowable drawdown
    
    if current_balance >= drawdown_limit:  # Only trade if the current balance is above drawdown limit
        for ticker in tickers:
            # Buy signal
            if signals[ticker]['positions'].iloc[i] == 1.0:
                if cash >= (signals[ticker]['price'].iloc[i] + fee_per_trade):
                    cash -= (signals[ticker]['price'].iloc[i] + fee_per_trade)
                    portfolio[ticker] += 1
                    trade_count += 1

            # Sell signal
            elif signals[ticker]['positions'].iloc[i] == -1.0:
                if portfolio[ticker] > 0:
                    cash += (signals[ticker]['price'].iloc[i] - fee_per_trade)
                    portfolio[ticker] -= 1
                    trade_count += 1
                    if cash > initial_balance:
                        winning_trades += 1

In [47]:
# Final account balance
final_balance = cash
for ticker in portfolio:
    final_balance += portfolio[ticker] * df['Adj Close'][ticker].iloc[i]

print(f"Final balance: ${final_balance:.2f}")
print(f"Trade Count: {trade_count}")
print(f"Winning Trades: {winning_trades}")
print(f"Winning Rate: {winning_trades / trade_count * 100 if trade_count > 0 else 0:.2f}%")


Final balance: $97043.06
Trade Count: 576
Winning Trades: 0
Winning Rate: 0.00%
